# Compressa

Compressa — это платформа, которая предлагает...

In [2]:
from langchain_compressa import CompressaEmbeddings
from langchain_compressa import ChatCompressa
from langchain_compressa import CompressaRerank

## Установка

In [ ]:
# установка пакета
!pip install langchain-compressa

## Настройка переменных окружения

Убедитесь, что у вас установлена следующая переменная окружения:

- COMPRESSA_API_KEY

In [1]:
import os
os.environ["COMPRESSA_API_KEY"] = "ваш_ключ_тут"

## Пример реализации RAG пайплайна с помощью Langchain Compressa

RAG (Retrieval Augmented Generation) - это метод работы с большими языковыми моделями, когда в контекст запроса к языковой модели 
программно добавляется дополнительная информация, на основе которой языковая модель может дать пользователю более полный и точный ответ.

In [ ]:
#установите дополнительные необходимые пакеты
!pip install langchain langchain_core langchain_community langchain_chroma bs4

In [ ]:
import os
from langchain_compressa import CompressaEmbeddings, ChatCompressa, CompressaRerank
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma

In [ ]:
COMPRESSA_API_KEY = os.getenv('COMPRESSA_API_KEY')

In [ ]:
#определим llm и embedding
compressa_embedding = CompressaEmbeddings(api_key=COMPRESSA_API_KEY)
llm = ChatCompressa(api_key=COMPRESSA_API_KEY)

In [ ]:
#определим загрузчик документов и получим документы langchain.
#здесь может быть использован любой из доступных загрузчиков.
loader = WebBaseLoader("https://ru.wikipedia.org/wiki/Архитектура_фон_Неймана")
docs = loader.load()

In [ ]:
#определим text_splitter и разобъём документы на чанки
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
#определим text_splitter и разобъём документы на чанки
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
#загрузим чанки документов в vectorstore и определим retriever
vectorstore = Chroma.from_documents(documents=all_splits, embedding=compressa_embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
#определим реранкер для использования в цепочке после извлечения документов
compressor = CompressaRerank(api_key=COMPRESSA_API_KEY)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
#определим Prompt для получения ответов на вопросы пользователя исходя из только контекста а не предыдущих знаний
system_template = f"""Ты помощник по вопросам-ответам. 
Используй следующую контекстную информацию, чтобы ответить на вопрос. 
Если в контексте нет ответа, ответь 'Не знаю ответа на вопрос'. 
Используй максимум три предложения и будь точным но кратким."""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", """Контекстная информация:

        {context}
        
        Вопрос: {input}		
    """),
])

In [ ]:
#зададим 2 цепочки: для извлечения и переранжирования документов по вопросу и для получения итогового ответа
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(compression_retriever, question_answer_chain)

In [ ]:
#теперь можем задавать вопрос
answ = rag_chain.invoke({"input": "Какое узкое место у архитектуры фон Неймана?"})
print(answ["answer"])